# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

# Find the ground state using direct minimisation (always using SCF is boring ...)
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.123490e+02     1.817351e+00
 * time: 0.08766603469848633
     1     1.176138e+01     1.026691e+00
 * time: 0.8908669948577881
     2    -1.138019e+01     1.220392e+00
 * time: 1.0475349426269531
     3    -3.363751e+01     8.682890e-01
 * time: 1.189640998840332
     4    -4.726716e+01     5.896484e-01
 * time: 1.3494930267333984
     5    -5.672952e+01     2.320052e-01
 * time: 1.5007739067077637
     6    -5.964552e+01     1.294336e-01
 * time: 1.6098220348358154
     7    -6.080857e+01     5.464929e-02
 * time: 1.7257938385009766
     8    -6.126068e+01     6.420213e-02
 * time: 1.8339319229125977
     9    -6.153738e+01     4.473067e-02
 * time: 1.9499969482421875
    10    -6.175111e+01     2.982199e-02
 * time: 2.059558868408203
    11    -6.190782e+01     1.931749e-02
 * time: 2.176745891571045
    12    -6.198426e+01     1.795725e-02
 * time: 2.286149024963379
    13    -6.205688e+01     1.632705e-02
 * time: 2.4008560180664

In [4]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671065
    AtomicLocal         -18.8557668
    AtomicNonlocal      14.8522641
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485369 
    Xc                  -19.3336819

    total               -62.261666461764
